In [53]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from binascii import hexlify, unhexlify



def hex_to_bits(hex_string):
    # Convert hex to binary
    binary_representation = bin(int(hex_string, 16))[2:]
    
    # Ensure leading zeros are included
    length = len(hex_string) * 4  # Each hex digit represents 4 bits
    return binary_representation.zfill(length)

def string_to_bits(input_string):
    return ''.join(format(ord(char), '08b') for char in input_string)


def bits_to_string(bits):
    return ''.join(chr(int(bits[i:i+8], 2)) for i in range(0, len(bits), 8))


def encrypt_string(input_string, key):
    # Convert string to bytes
    input_bytes = input_string.encode('utf-8')

    # Pad the data
    padded_data = pad(input_bytes, AES.block_size)

    # Generate a random IV
    iv = get_random_bytes(AES.block_size)

    # Create AES cipher object
    cipher = AES.new(key, AES.MODE_CBC, iv)

    # Encrypt the data
    ciphertext = cipher.encrypt(padded_data)

    # Convert the ciphertext to bit representation
    ciphertext_bits = hex_to_bits(hexlify(ciphertext).decode('utf-8'))

    return iv, ciphertext_bits

# Example usage:
key = get_random_bytes(AES.block_size)  # Generate a random key
input_string = "123456789abcdef"

# Step 1: Convert string to bits
input_bits = string_to_bits(input_string)
print("Input String (in bits):", input_bits)

# Step 2: Encrypt the string
iv, ciphertext_bits = encrypt_string(input_string, key)
print("Ciphertext (in bits):", ciphertext_bits)

# Step 3: Decrypt the ciphertext
# decrypted_string = decrypt_bits(ciphertext_bits, key, iv)
# print("Decrypted String:", decrypted_string)

Input String (in bits): 001100010011001000110011001101000011010100110110001101110011100000111001011000010110001001100011011001000110010101100110
Ciphertext (in bits): 01011001000100100000010100100111101110010110010010000111000100100110101000101001001010011101110011100100010000101100101011011101


In [54]:
len(input_bits),len(ciphertext_bits)

(120, 128)

In [55]:
import nltk
from nltk.corpus import genesis

In [56]:
genesis_words = genesis.words()

In [57]:
def filter_letter(let):

    if ord('a') <= ord(let) <= ord('z'):
        return True

    else:
        return False

In [58]:
def make_blocks(word_list, block_size):
    word_str = "".join(word_list)
    new_str = ""
    blocked_arr = []

    for i in range(0, len(word_str)):

        let = word_str[i].lower()
        if filter_letter(let):
            new_str += let
        else:
            pass

    del word_str

    for i in range(0, len(new_str), block_size):
        block = new_str[i:i+block_size]

        if len(block) == block_size:
            blocked_arr.append(block)
        else:
            diff = block_size - len(block)
            padded_block = block + "x" * diff
            blocked_arr.append(padded_block)

    return blocked_arr

In [59]:
pt_blocks = make_blocks(genesis_words, 8)

In [60]:
first_block = pt_blocks[0]

In [61]:
len(string_to_bits(first_block))

64

In [62]:
_,ct_bits = encrypt_string(first_block,key)
len(ct_bits)

128

In [63]:
len(hex_to_bits(hexlify(key)))

128

In [64]:
pt_bits = []
for pt in pt_blocks:
    pt_bits.append(string_to_bits(pt))


In [65]:
ct_bits = []

for pt in pt_blocks:
    # ct is in bits
    _,ct = encrypt_string(pt,key)
    ct_bits.append(ct)


# Converting to a dataset

In [66]:
import numpy as np

In [67]:
for bits in pt_bits:
    if bits.isnumeric():
        pass
    else:
        print("Don't Panic")
for bits in ct_bits:
    if bits.isnumeric():
        pass
    else:
        print("Don't Panic")

In [74]:
ptb_array = []
ctb_array = []

# 3 is ending token, 2 is starting token
for ptb in pt_bits:
    x = [int(i) for i in list(ptb)] + [3]
    ptb_array.append(x)

for ctb in ct_bits:
    x = [2] + [int(i) for i in list(ctb)] + [3]
    ctb_array.append(x)


    

In [75]:
ptb_array = np.array(ptb_array)
ctb_array = np.array(ctb_array)
# for a fixed key

In [76]:
ptb_array.shape,ctb_array.shape

((134339, 65), (134339, 130))

# Making the model

In [71]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer

In [78]:
unique_chars = set()

for word in ptb_array:

    for let in word:

        if not let in unique_chars:
            unique_chars.add(let)


print(len(unique_chars))

3


In [72]:
batch_size = 64
num_samples = 134339
latent_dim = 256
epochs = 15

In [ ]:
num_encoder_tokens = 2
num_decoder_tokens = 3